## Simulation Framework 

In [1]:
import import_ipynb
import math 
import importlib
import random
import copy

# import functions from model_evaluation.ipynb  
import model_evaluation 

importing Jupyter notebook from model_evaluation.ipynb


In [2]:
# simulation parameters 

# number of groups
n_groups = 4 

# number of agents 
n_agents = 4

# Iterations for group distributions 
iterations = 100

### Group Distributions 

In [3]:
# create random distributions 
def distribute_agents(n_agents, n_groups):
    """
    This function randomly selects a subset of group indices for each agent. The number of groups for each agent is randomly chosen between 1 and the total number of groups.
    :returns (list of lists): Each inner list represents the agents assigned to a particular group.
    """
    group_dist = [[] for _ in range(n_groups)]

    for agent in range(n_agents):
        groups = random.sample(range(n_groups), random.randint(1, n_groups))
        for group in groups:
            group_dist[group].append(agent)

    return group_dist

In [4]:
# Create random group distributions with a gradual transition from one to the other extreme 
def gradual_distribution_transition(n_agents, n_groups, iterations):
    """
    This function starts with a distribution where each group contains only one agent and gradually modifies it in each iteration.
    :returns (list of lists): Each inner list represents the agents assigned to a particular group.
    """
    # Initial distribution: Each group contains only one agent
    group_dist = [[agent] for agent in range(n_agents)]

    # List to store distributions at each iteration
    distributions = [copy.deepcopy(group_dist)]

    for _ in range(iterations):  # Start from iteration 1
        # Randomly select a group
        source_group = random.randint(0, n_groups - 1)
        
        # Randomly select a destination group different from the source group
        possible_destination_groups = [group for group in range(n_groups) if group != source_group]
        destination_group = random.choice(possible_destination_groups)

        # Randomly select an agent from the source group
        if group_dist[source_group]:
            agent_to_move = random.choice(group_dist[source_group])

            # Move the agent to the destination group if not already present
            if agent_to_move not in group_dist[destination_group]:
                group_dist[destination_group].append(agent_to_move)
            else:
                continue

        # Save the current distribution (deep copy to avoid referencing the same object)
        distributions.append(copy.deepcopy(group_dist))

    return distributions

In [27]:
result_group_distributions = gradual_distribution_transition(n_agents=n_agents, n_groups=n_groups, iterations=iterations)
# print(result_group_distributions)

In [28]:
# Custom Group Distributions 
# result_group_distributions = [[[0], [1]], [[0, 1], [1]], [[0, 1], [1, 0]]]
# result_group_distributions = [[[0], [1]], [[0, 1], [1, 0]]]
result_group_distributions = [[[0, 1], [1]], [[0, 1], [1, 0]]]
# print(result_group_distributions)

### Vote Distribution 

In [17]:
# define fixed vote distribution 
def vote_distribution_fixed(n_agents, votes_agent):
    return [votes_agent] * n_agents

In [18]:
vote_dist = vote_distribution_fixed(n_agents=2, votes_agent=5)
print(vote_dist)

[5, 5]


### Group Membership

In [19]:
list_result_group_memberships = []
for i in range(len(result_group_distributions)): 
    result_group_memberships = model_evaluation.group_memberships(groups=result_group_distributions[i], votes=vote_dist)
    list_result_group_memberships.append(result_group_memberships)

print(list_result_group_memberships)

[[[0], [0, 1]], [[0, 1], [0, 1]]]


In [30]:
result_commom_group = model_evaluation.common_group(1,0, memberships_group=list_result_group_memberships[1])
# print(result_commom_group)

agent_i: 1, agent_j: 0
group_memberships[agent_i]: [0, 1], type: <class 'list'>
group_memberships[agent_j]: [0, 1], type: <class 'list'>


In [31]:
result_K = model_evaluation.K(agent_i=0, group=result_group_distributions[1][1], votes=vote_dist, memberships_group=list_result_group_memberships[1])
# print(result_K)

### Model Terms 

In [32]:
# first term 
dict_result_group_first_term = {}
list_result_aggregated_weight_term_1 = []
for i in range(len(result_group_distributions)):  # len(result_group_distributions) == len(list_result_group_memberships)
    result_groups, result_aggregate_weight = model_evaluation.first_term(groups=result_group_distributions[i], votes=vote_dist, memberships_group=list_result_group_memberships[i])
    dict_result_group_first_term[i] = {'individual_contributions': result_groups}
    list_result_aggregated_weight_term_1.append(result_aggregate_weight)

# print(list_result_aggregated_weight_term_1)
# print(dict_result_group_first_term)

In [33]:
# interaction terms
dict_result_individual_interaction_terms = {}
list_result_aggregated_weight_interaction_terms = []
for i in range(len(result_group_distributions)):  # len(result_group_distributions) == len(list_result_group_memberships)
    result_individual, result_aggregated = model_evaluation.interaction_terms(groups=result_group_distributions[i], memberships_group=list_result_group_memberships[i], votes=vote_dist)
    dict_result_individual_interaction_terms[i] = {'individual_contributions': result_individual}
    list_result_aggregated_weight_interaction_terms.append(result_aggregated)

# print(dict_result_individual_interaction_terms)
# print(list_result_aggregated_weight_interaction_terms)   


agent_i: 0, agent_j: 1
group_memberships[agent_i]: [0], type: <class 'list'>
group_memberships[agent_j]: [0, 1], type: <class 'list'>
agent_i: 0, agent_j: 1
group_memberships[agent_i]: [0], type: <class 'list'>
group_memberships[agent_j]: [0, 1], type: <class 'list'>


In [34]:
# print(math.sqrt(20))

### Plurality and QV Scorese 

In [25]:
list_plurality_scores = []
for i in range(len(list_result_aggregated_weight_term_1)):   
    plurality_score = model_evaluation.connection_oriented_cluster_match(term_1=list_result_aggregated_weight_term_1[i], interaction_terms=list_result_aggregated_weight_interaction_terms[i])
    list_plurality_scores.append(plurality_score)

print(list_plurality_scores)

[3.724645398720181, 3.804226065180614]


In [26]:
result_quadratic_voting, result_sum_quadratic_votes = model_evaluation.quadratic_voting(votes=vote_dist)
print(result_quadratic_voting)
print(result_sum_quadratic_votes)

{0: 2.23606797749979, 1: 2.23606797749979}
4.47213595499958


### Jaccard Index 

In [82]:
global_jaccard_index = []
for i in range(len(result_group_distributions)):
    pairwise_jaccard_matrix = model_evaluation.pairwise_jaccard_similarity(groups=result_group_distributions[i])
    # print("Pairwise Jaccard Similarity Matrix:")
    # for row in pairwise_jaccard_matrix:
    #    print(row)

    global_similarity = model_evaluation.global_jaccard_similarity(pairwise_jaccard_matrix)
    global_jaccard_index.append(global_similarity)

print(global_jaccard_index)


[0.0, 0.08333333333333333, 0.13888888888888887, 0.16666666666666666, 0.27777777777777773, 0.3055555555555555, 0.38888888888888884, 0.5277777777777778, 0.5694444444444444, 0.625, 0.75, 0.875, 1.0]
